In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
gdf_residentials = gpd.read_file('../gangnamGu-building-residential.geojson', encoding='utf8', dtype={'USABILITY':str})

In [3]:
gdf_hexagons = gpd.read_file('../hexagons_filtered.geojson')

## Calculate the number of residential buildings and total building area of residential buildings

In [4]:
# 01000 단독주택
# 01001 단독주택
single = ['01000','01001']

# 01002 다중주택
# 01003 다가구주택
# 02003 다세대주택
multi = ['01002','01003','02003']

# 02000 공동주택
# 02001 아파트
# 02002 연립주택
flat = ['02000','02001','02002']

# 10202 오피스텔
mxd = ['10202']

# 01004 공관
# 02007 기숙사 
etc = ['01004','02007']

In [5]:
gdf_residentials['USABILITY'].unique()

array(['01000', '02000'], dtype=object)

In [6]:
gdf_single = gdf_residentials.loc[gdf_residentials['USABILITY']=='01000']
gdf_multi = gdf_residentials.loc[gdf_residentials['USABILITY']=='02000']

In [7]:
gdf_residentials.shape

(11833, 15)

In [8]:
gdf_single.shape

(6484, 15)

In [9]:
gdf_multi.shape

(5349, 15)

In [10]:
def count_buildings(polygon, target_buildings):
    '''
    This function is for making a list of buildings that is in a hexagon area.
    -----
    input:
    polygon (Shapely.geometry.Polygon): a hexagon polygon 
    -----
    output:
    precise_matches_list (list): a list of residential buildings
    
    '''
    try:
        buildings_sindex = target_buildings.sindex
        possible_matches_index = list(buildings_sindex.intersection(polygon.bounds))
        possible_matches = target_buildings.iloc[possible_matches_index]
        # extract the list of buildings that is in a hexagon
        precise_matches_list = possible_matches[possible_matches.intersects(polygon)]['UFID'].tolist()
    except:
        # if there is no buildings that matched within a hexagon, return empty list
        precise_matches_list = []
    return precise_matches_list

In [11]:
gdf_hexagons['single_houses'] = gdf_hexagons['geometry'].apply(lambda x: count_buildings(x,gdf_single))

In [12]:
gdf_hexagons['multi_houses'] = gdf_hexagons['geometry'].apply(lambda x: count_buildings(x,gdf_multi))

In [13]:
gdf_residentials.columns

Index(['USABILITY', 'HEIGHT', 'PNU', 'GRND_FLR', 'UGRND_FLR', 'DONG_NM',
       'COL_ADM_SE', 'UFID', 'BLDG_PNU', 'BD_MGT_SN', 'ARCHAREA', 'TOTALAREA',
       'BC_RAT', 'VL_RAT', 'geometry'],
      dtype='object')

In [19]:
def sum_archarea(house_list):
    total = gdf_residentials.loc[gdf_residentials['UFID'].isin(house_list)]['ARCHAREA'].sum()
    return total

In [20]:
gdf_hexagons['single_archarea'] = gdf_hexagons['single_houses'].apply(lambda x: sum_archarea(x))

In [21]:
gdf_hexagons['multi_archarea'] = gdf_hexagons['multi_houses'].apply(lambda x: sum_archarea(x))

In [23]:
gdf_hexagons['single_count'] = gdf_hexagons['single_houses'].apply(lambda x: len(x))
gdf_hexagons['multi_count'] = gdf_hexagons['multi_houses'].apply(lambda x: len(x))

In [25]:
gdf_hexagons = gdf_hexagons.drop(['single_houses','multi_houses'], axis=1)

##  Count the number of public bike stations

In [26]:
gdf_bike = gpd.read_file('../publicBikeStation.geojson')

In [29]:
gdf_bike.head()

,stationid,gu,address,y,x,lcd_number,qr_number,type,geometry
0,1695,???,??? ??? 14,37.623417,127.066933,NaN,10.0,QR,POINT (127.06693 37.62342)
1,2301,???,????? ??? ???? 134,37.524071,127.021790,10.0,NaN,LCD,POINT (127.02179 37.52407)
2,2302,???,????? ??? ???? ?? 102,37.505581,127.024277,10.0,NaN,LCD,POINT (127.02428 37.50558)
3,2303,???,????? ??? ??? ?? 102,37.511517,127.021477,15.0,NaN,LCD,POINT (127.02148 37.51152)
4,2304,???,????? ??? ??? 626,37.512527,127.035835,10.0,NaN,LCD,POINT (127.03584 37.51253)


In [33]:
def count_stations(polygon, stations):
    try:
        station_sindex = stations.sindex
        possible_matches_index = list(station_sindex.intersection(polygon.bounds))
        possible_matches = stations.iloc[possible_matches_index]
        # extract the list of buildings that is in a hexagon
        precise_matches_list = possible_matches[possible_matches.intersects(polygon)]['stationid'].tolist()
    except:
        # if there is no station that matched within a hexagon, return empty list
        precise_matches_list = []
    return len(precise_matches_list)                        

In [34]:
gdf_hexagons['count_bike_station'] = gdf_hexagons['geometry'].apply(lambda x:count_stations(x,gdf_bike))

## Count the number of bus lines

In [42]:
gdf_bus_stop = gpd.read_file('../seoulBusStop.geojson', dtype={'ARS-ID':str})

In [43]:
gdf_bus_stop.head()

,표준ID,ARS-ID,정류장명,X좌표,Y좌표,geometry
0,100000001,1001,종로2가사거리,126.987750,37.569765,POINT (126.98775 37.56977)
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,POINT (126.99657 37.57918)
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671,POINT (126.99834 37.58267)
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,POINT (126.98761 37.56858)
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,POINT (127.00174 37.58624)


In [50]:
df_bus_line = pd.read_csv('../seoulBusLine.csv')

In [51]:
df_bus_line.head()

,노선ID,노선명,순번,구간ID,정류소ID,ARS-ID,정류소명,X좌표,Y좌표
0,100100124,0017,1,0,102000271,3689,청암자이아파트,126.946517,37.534363
1,100100124,0017,2,102700549,102000204,3298,청암동강변삼성아파트,126.949304,37.533961
2,100100124,0017,3,102700550,102000227,3321,청심경로당,126.950449,37.533744
3,100100124,0017,4,102700551,102000210,3304,원효2동주민센터,126.950904,37.534278
4,100100124,0017,5,102700552,102000212,3306,산천동,126.953984,37.535420


In [46]:
def get_bus_stations(polygon, stations):
    try:
        station_sindex = stations.sindex
        possible_matches_index = list(station_sindex.intersection(polygon.bounds))
        possible_matches = stations.iloc[possible_matches_index]
        # extract the list of buildings that is in a hexagon
        precise_matches_list = possible_matches[possible_matches.intersects(polygon)]['표준ID'].tolist()
    except:
        # if there is no station that matched within a hexagon, return empty list
        precise_matches_list = []
    return precise_matches_list   

In [47]:
gdf_hexagons['bus_stops'] = gdf_hexagons['geometry'].apply(lambda x: get_bus_stations(x,gdf_bus))

In [60]:
def count_bus_lines(bus_stops, lines):
    return len(lines[lines['정류소ID'].isin(bus_stops)]['노선ID'].unique().tolist())

In [61]:
gdf_hexagons['count_bus_line'] = gdf_hexagons['bus_stops'].apply(lambda x: count_bus_lines(x,df_bus_line))

In [62]:
gdf_hexagons.head()

,index,x,y,geometry,single_archarea,multi_archarea,single_count,multi_count,count_bike_station,bus_stops,count_bus_line
0,8830e1ca31fffff,127.046889,37.500817,"POLYGON ((127.05109 37.49886, 127.05058 37.503...",2769.01,41236.2603,20,94,3,"[122000602, 122000202, 122000174, 122000603, 1...",18
1,8830e1cae9fffff,127.061157,37.514724,"POLYGON ((127.06536 37.51277, 127.06485 37.517...",5531.18,32655.2480,49,128,3,"[122000095, 122900015, 122000088, 122000728, 1...",21
2,8830e1ca23fffff,127.035810,37.493646,"POLYGON ((127.04001 37.49169, 127.03950 37.496...",39876.47,46344.5308,328,215,1,"[122900045, 122000210, 122000187, 122000188, 1...",8
3,8830e1c853fffff,127.072123,37.489082,"POLYGON ((127.07633 37.48713, 127.07582 37.491...",0.00,41341.5000,0,82,3,"[122000359, 122000371, 122000385, 122000360, 1...",13
4,8830e1c81bfffff,127.084741,37.483211,"POLYGON ((127.08895 37.48125, 127.08844 37.485...",0.00,301466.5070,0,80,1,"[122900001, 122000279, 122000280, 122000277, 1...",10


In [63]:
gdf_hexagons = gdf_hexagons.drop('bus_stops', axis=1)

In [64]:
gdf_hexagons.head()

,index,x,y,geometry,single_archarea,multi_archarea,single_count,multi_count,count_bike_station,count_bus_line
0,8830e1ca31fffff,127.046889,37.500817,"POLYGON ((127.05109 37.49886, 127.05058 37.503...",2769.01,41236.2603,20,94,3,18
1,8830e1cae9fffff,127.061157,37.514724,"POLYGON ((127.06536 37.51277, 127.06485 37.517...",5531.18,32655.2480,49,128,3,21
2,8830e1ca23fffff,127.035810,37.493646,"POLYGON ((127.04001 37.49169, 127.03950 37.496...",39876.47,46344.5308,328,215,1,8
3,8830e1c853fffff,127.072123,37.489082,"POLYGON ((127.07633 37.48713, 127.07582 37.491...",0.00,41341.5000,0,82,3,13
4,8830e1c81bfffff,127.084741,37.483211,"POLYGON ((127.08895 37.48125, 127.08844 37.485...",0.00,301466.5070,0,80,1,10


## Count subway lines

In [66]:
gdf_subway_station = gpd.read_file('../seoulSubwayStation.geojson')

In [67]:
gdf_subway_station.head()

,stationID,stationName,Line,y,x,geometry
0,2818,????,8,37.492522,127.118234,POINT (127.11823 37.49252)
1,340,????,3,37.492522,127.118234,POINT (127.11823 37.49252)
2,2535,??3?,5,37.571607,126.991806,POINT (126.99181 37.57161)
3,319,??3?,3,37.571607,126.991806,POINT (126.99181 37.57161)
4,153,??3?,1,37.571607,126.991806,POINT (126.99181 37.57161)


In [68]:
def count_subway_lines(polygon, stations):
    try:
        station_sindex = stations.sindex
        possible_matches_index = list(station_sindex.intersection(polygon.bounds))
        possible_matches = stations.iloc[possible_matches_index]
        # extract the list of buildings that is in a hexagon
        precise_matches_list = possible_matches[possible_matches.intersects(polygon)]['Line'].unique().tolist()
    except:
        # if there is no station that matched within a hexagon, return empty list
        precise_matches_list = []
    return len(precise_matches_list)  

In [70]:
gdf_hexagons['count_subway_line'] = gdf_hexagons['geometry'].apply(lambda x: count_subway_lines(x,gdf_subway_station))

## Calculate accessibility score

In [73]:
max_bus_line = gdf_hexagons['count_bus_line'].max()
max_bike_station = gdf_hexagons['count_bike_station'].max()
max_subway_station = gdf_hexagons['count_subway_line'].max()

In [74]:
print(max_bus_line, max_bike_station, max_subway_station)

35 4 2


In [75]:
gdf_hexagons.head()

,index,x,y,geometry,single_archarea,multi_archarea,single_count,multi_count,count_bike_station,count_bus_line,count_subway_line
0,8830e1ca31fffff,127.046889,37.500817,"POLYGON ((127.05109 37.49886, 127.05058 37.503...",2769.01,41236.2603,20,94,3,18,0
1,8830e1cae9fffff,127.061157,37.514724,"POLYGON ((127.06536 37.51277, 127.06485 37.517...",5531.18,32655.2480,49,128,3,21,1
2,8830e1ca23fffff,127.035810,37.493646,"POLYGON ((127.04001 37.49169, 127.03950 37.496...",39876.47,46344.5308,328,215,1,8,0
3,8830e1c853fffff,127.072123,37.489082,"POLYGON ((127.07633 37.48713, 127.07582 37.491...",0.00,41341.5000,0,82,3,13,1
4,8830e1c81bfffff,127.084741,37.483211,"POLYGON ((127.08895 37.48125, 127.08844 37.485...",0.00,301466.5070,0,80,1,10,1


In [76]:
gdf_hexagons['score_bus'] = (gdf_hexagons['count_bus_line']/max_bus_line) * 5
gdf_hexagons['score_bike'] = (gdf_hexagons['count_bike_station']/max_bike_station) * 5
gdf_hexagons['score_bus'] = (gdf_hexagons['count_subway_line']/max_subway_station) * 5

In [77]:
gdf_hexagons.to_file('../hexagon_with_data.geojson', driver='GeoJSON', encoding='utf8')